In [2]:
import pandas as pd
import numpy as np
import requests
import regex as re
import geopandas, shapely
import json
from functools import reduce
import operator
from pymongo import MongoClient,GEOSPHERE

In [158]:
# Set connection with MongoDB
conn = MongoClient("localhost:27017")
db = conn.get_database("final_project")
p = db.get_collection("places")
h = db.get_collection("homes")

In [159]:
p.create_index([("geometry",GEOSPHERE)])
h.create_index([("geometry",GEOSPHERE)])

'geometry_2dsphere'

In [161]:
def insert_object(df, collection):
    '''
    This function inserts all the information from the dataframe we created with the previous function as a Mongodb object.
    This object will have two field: title (name of the film or show) and reviews (array with all the reviews users have made)
    '''

    collection.insert_many(df.to_dict('records'))

    return "Data succesfully added"

In [154]:
insert_object(restaurants, p)

'Data succesfully added'

### Getting home data and cleaning it

In [6]:
# Download JSON file 
data = json.load(open("data\\data_vivienda.json", 'r', encoding='utf-8'))

In [5]:
data_vivienda = pd.DataFrame(data)
data_vivienda.columns

Index(['propertyCode', 'thumbnail', 'externalReference', 'numPhotos', 'floor',
       'price', 'propertyType', 'operation', 'size', 'exterior', 'rooms',
       'bathrooms', 'address', 'province', 'municipality', 'district',
       'country', 'neighborhood', 'latitude', 'longitude', 'showAddress',
       'url', 'hasVideo', 'status', 'newDevelopment', 'hasLift', 'priceByArea',
       'detailedType', 'suggestedTexts', 'hasPlan', 'has3DTour', 'has360',
       'parkingSpace', 'newDevelopmentFinished'],
      dtype='object')

In [6]:
data_vivienda = data_vivienda.drop(["thumbnail","propertyCode", "externalReference", "numPhotos", "hasVideo", "priceByArea", "detailedType", "country", "suggestedTexts", "hasPlan", "has3DTour", "has360", "newDevelopmentFinished"], axis = 1)

In [7]:
# I'm only interested in those houses inside Madrid, so I'm going to delete all observations that belong to another municipality
data_vivienda = data_vivienda[data_vivienda["municipality"]=="Madrid"]

In [8]:
data_vivienda.dtypes

floor              object
price             float64
propertyType       object
operation          object
size              float64
exterior             bool
rooms               int64
bathrooms           int64
address            object
province           object
municipality       object
district           object
neighborhood       object
latitude          float64
longitude         float64
showAddress          bool
url                object
status             object
newDevelopment       bool
hasLift            object
parkingSpace       object
dtype: object

In [9]:
data_vivienda.isnull().sum()
# Parkingspace has too many null values, and as we cannot fill the with a proper value the best option is to drop the whole column (explain in API the reason for the lack of information)
# Let's check the floor, status and hasLift variables

floor              108
price                0
propertyType         0
operation            0
size                 0
exterior             0
rooms                0
bathrooms            0
address              0
province             0
municipality         0
district             0
neighborhood         0
latitude             0
longitude            0
showAddress          0
url                  0
status             556
newDevelopment       0
hasLift            154
parkingSpace      6018
dtype: int64

In [10]:
data_vivienda = data_vivienda.drop("parkingSpace", axis = 1)

In [74]:
data_vivienda.status.value_counts()
# In this case we are going to fill the null values with 'unspecified'

good              8000
renew               24
newdevelopment       8
Name: status, dtype: int64

In [11]:
# There a couple of values that are not numbers, so I'm going to change them to a value that can be valid. In this case '0', as after looking a Idelista we can assume they are street level homes
data_vivienda.floor = data_vivienda.floor.replace(["bj","en","st","ss"], "0", regex = True)

In [12]:
data_vivienda.status = data_vivienda.status.fillna('unspecified')

In [13]:
data_vivienda.hasLift = data_vivienda.hasLift.fillna("unspecified")

In [14]:
data_vivienda = data_vivienda.dropna(how = 'any')

In [68]:
def geo_frame(df):
    df = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.latitude, df.longitude))
    df['geometry']=df['geometry'].apply(lambda x:shapely.geometry.mapping(x))
    return df

In [17]:
data_vivienda = geo_frame(data_vivienda)

c:\users\juanp\anaconda3\envs\ironhack\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [16]:
# As I have limited resources, and the number of queries I can do in all APIs is limited, I'm going to focus on three district with the more number of houses available

test = data_vivienda[data_vivienda.groupby('district').district.transform('count')>800]

In [17]:
test.district.value_counts()

Centro       1741
Salamanca    1727
Chamberí      972
Chamartín     813
Name: district, dtype: int64

In [414]:
insert_object(test, h)

'Data succesfully added'

In [18]:
test.to_csv("data\\test.csv")

### Google API

In [97]:
my_key = open("keys\\google_key.txt").readlines()[0]

In [7]:
response = requests.get(url)

#### Find places with google API

In [98]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

In [99]:
location = '40.446067,-3.691247' # I have picked Nuevos Minesterios as center of all queries
radius = '10000'
keyword = "bus%20stop"
page_token = my_key
url = f"{base_url}location={location}&radius={radius}&keyword={keyword}&key={my_key}"

### FOURSQUARE API

In [16]:
client_id = open("keys\\foursquare_key.txt").readlines()[0]
client_secret = open("keys\\foursquare_key.txt").readlines()[1]
version = '20180323'

In [8]:
def extract(url):
    
    results = requests.get(url)

    code = json.loads(results.text)

    decoding = code.get("response")

    decoded = decoding.get("groups")[0]

    return decoded.get("items")

In [278]:
def getFromDict(diccionario,objeto):
    return reduce(operator.getitem,objeto,diccionario)

def places_df(frame,category):
    
    nombre = ["venue","name"]
    latitud = ["venue", "location","lat"]
    longitud = ["venue","location","lng"]
    barrio = ["venue", "location", "neighborhood"]
    tipo = ["venue", "categories", "pluralName"]
    x = []
    
    for diccionario in frame:
        lista = {}
        lista["name"] = getFromDict(diccionario,nombre)
        lista["longitude"] = getFromDict(diccionario,longitud)
        lista["latitude"] = getFromDict(diccionario,latitud)

        
        x.append(lista)
        
        df = pd.DataFrame(x)

    c = [category for _ in range(len(frame))]
        
    df["category"] = c
    
    return df

In [285]:
location = "40.462781, -3.686737"
limit = 10
cat_id = "4bf58dd8d48988d116941735"

In [17]:
def locations(radius, limit,location, cat_id, category ):
    url = f'https://api.foursquare.com/v2/venues/explore?&client_id={client_id}&client_secret={client_secret}&v={version}&limit=200&ll={location}&radius={radius}&limit={limit}&categoryId={cat_id}'
    x = extract(url)
    return places_df(x, category)

### Find establishments using Foursquare to populate database

In [274]:
# IDs
nightlife = "4d4b7105d754a06376d81259"
gym = "4bf58dd8d48988d175941735"
park = "4bf58dd8d48988d163941735"
hospital = "4bf58dd8d48988d196941735"
elementary_school = "4f4533804b9074f6e4fb0105"
high_school = "4bf58dd8d48988d13d941735"
middle_school = "4f4533814b9074f6e4fb0106"
clothing_store = "4bf58dd8d48988d103951735"
daycare = "4f4532974b9074f6e4fb0104"
grocery_store = "4bf58dd8d48988d118951735"
supermarket = "52f2ab2ebcbc57f1066b8b46"
pharmacy = "4bf58dd8d48988d10f951735"
general_entertainment = "4d4b7104d754a06370d81259"
bar = "4bf58dd8d48988d116941735"
metro_station = "4bf58dd8d48988d1fd931735"
train_station = "4bf58dd8d48988d129951735"
bus_stop = "52f2ab2ebcbc57f1066b8b4f"
elementary_school = "4f4533804b9074f6e4fb0105"
high_school = "4bf58dd8d48988d13d941735"
restaurant = "4d4b7105d754a06374d81259"

#### Use of queries_complete function 

In [188]:
def queries(loc):

    return locations(5000, 100, loc, restaurants, "restaurant")

def queries_complete(lista, column):
    result = list(map(queries, lista))
    x = pd.DataFrame(result[0])
    y = pd.DataFrame(result[1])
    w = pd.DataFrame(result[2])

    return pd.concat([x,y,w]).drop_duplicates(subset=column, keep='first')
        

In [180]:
lista = [("40.462781, -3.686737"), ("40.432751, -3.683338"), ("40.412951, -3.707111")]

In [149]:
restaurants = queries_complete(lista, "name")

In [186]:
def queries(loc):

    return locations(5000, 100, loc, hospital, "medical_centre")

hospitals = queries_complete(lista, "name")

In [201]:
def queries(loc):

    return locations(5000, 100, loc, nightlife, "nightlife")

nightlife = queries_complete(lista, "name")

In [276]:
def queries(loc):

    return locations(5000, 5, loc, gym, "gym")

gym = queries_complete(lista, "latitude")

In [207]:
def queries(loc):

    return locations(5000, 100, loc, supermarket, "supermarket")
supermarket = queries_complete(lista, "latitude")

In [209]:
def queries(loc):

    return locations(5000, 100, loc, grocery_store, "supermarket")

grocery = queries_complete(lista, "latitude")

In [246]:
supermarkets = pd.concat([supermarket, grocery]).drop_duplicates(subset="latitude", keep='first')

In [212]:
def queries(loc):

    return locations(5000, 100, loc, general_entertainment, "general_entertainment")

ent = queries_complete(lista, "name")

In [214]:
def queries(loc):

    return locations(5000, 100, loc, pharmacy, "pharmacy")

pharmacy = queries_complete(lista, "latitude")

In [218]:
def queries(loc):

    return locations(5000, 100, loc, metro_station, "transport")

metro = queries_complete(lista, "name")

In [219]:
def queries(loc):

    return locations(5000, 100, loc, train_station, "transport")

train = queries_complete(lista, "name")

In [223]:
train = train[(train["name"].str.contains("Cercanías")) | train["name"].str.contains("Intercambiador") | train["name"].str.contains("Estación")]

In [227]:
train = train.drop(train[(train["name"].str.contains("Andén")) | train["name"].str.contains("Vía")| train["name"].str.contains("AVE") | train["name"].str.contains("VIP")].index)

In [254]:
transport = pd.concat([train, metro])

In [231]:
def queries(loc):

    return locations(5000, 100, loc, clothing_store, "clothing_store")

store = queries_complete(lista, "latitude")

In [233]:
def queries(loc):

    return locations(5000, 100, loc, park, "park")

parks = queries_complete(lista, "latitude")

In [237]:
parks = parks[(parks["name"].str.contains("Parque"))]

#### Joining dataframes 

In [538]:
total = pd.concat([conjunto, night, gyms, hosp, general, subs, ])

In [76]:
result = requests.get("https://datos.madrid.es/egob/catalogo/201132-0-museos.xml")

### Turn into geoframe and insert into MongoDB

In [ ]:
total = geo_frame(total)

In [541]:
insert_object(total, p)

'Data succesfully added'

#### Schools

In [69]:
schools = geo_frame(schools)

c:\users\juanp\anaconda3\envs\ironhack\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [197]:
insert_object(schools,p)

'Data succesfully added'

#### Restaurants

In [166]:
restaurants = geo_frame(restaurants)

c:\users\juanp\anaconda3\envs\ironhack\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [198]:
insert_object(restaurants,p)

'Data succesfully added'

#### Hospitals

In [190]:
hospitals = geo_frame(hospitals)

c:\users\juanp\anaconda3\envs\ironhack\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [199]:
insert_object(hospitals,p)

'Data succesfully added'

#### Nightlife

In [240]:
nightlife = geo_frame(nightlife)

c:\users\juanp\anaconda3\envs\ironhack\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [242]:
insert_object(nightlife, p)

'Data succesfully added'

#### Supermarkets and grocery stores

In [248]:
supermarkets = geo_frame(supermarkets)

c:\users\juanp\anaconda3\envs\ironhack\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [250]:
insert_object(supermarkets, p)

'Data succesfully added'

#### Parks

In [251]:
parks = geo_frame(parks)

c:\users\juanp\anaconda3\envs\ironhack\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [253]:
insert_object(parks, p)

'Data succesfully added'

#### Transport

In [255]:
transport = geo_frame(transport)

c:\users\juanp\anaconda3\envs\ironhack\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [256]:
insert_object(transport,p)

'Data succesfully added'

#### Clotjing store

In [257]:
store = geo_frame(store)

c:\users\juanp\anaconda3\envs\ironhack\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [258]:
insert_object(store, p)

'Data succesfully added'

#### General entertainment

In [259]:
ent = geo_frame(ent)

c:\users\juanp\anaconda3\envs\ironhack\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [260]:
insert_object(ent,p)

'Data succesfully added'

#### Pharmacy

In [261]:
pharmacy = geo_frame(pharmacy)

c:\users\juanp\anaconda3\envs\ironhack\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [263]:
insert_object(pharmacy,p)

'Data succesfully added'

#### Gym

In [264]:
gym = geo_frame(gym)

c:\users\juanp\anaconda3\envs\ironhack\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [265]:
insert_object(gym,p)

'Data succesfully added'